In [1]:
from header import *

D = 10
set_seed(3342432)

model1 = nsp.model.UnitaryRiemanGenerator(D, dtype=torch.float64)
model2 = nsp.model.UnitaryRiemanGenerator(D, dtype=torch.float64)
model3 = nsp.model.UnitaryRiemanGenerator(D, dtype=torch.float64)
model4 = nsp.model.UnitaryRiemanGenerator(D, dtype=torch.float64)
model = copy.deepcopy(model1)

X1 = model4.matrix().data.numpy() @ np.diag(np.random.rand(10)) @ model4.matrix().data.numpy().T
X2 = model3.matrix().data.numpy() @ np.diag(np.random.rand(10)) @ model3.matrix().data.numpy().T

In [2]:
X = np.kron(X1, X1)

In [3]:


# X = np.random.randn(D**2, D**2)
# X = (X + X.T)/2
# mask = np.abs(X)>=(1/(D))
# X[mask] = 0

loss = nsp.loss.MES(torch.Tensor(X), [D,D], inv = model1._inv)
cg1 = RiemanUnitaryCG(model, loss, lr = 0.1)
cg2 = RiemanUnitaryCG2([(model1, model2)], [loss])


# S, W = cg._riemannian_grad(model._params)

solver1 = UnitarySymmTs(RiemanUnitaryCG, model, loss, lr = 0.1)
solver2 = UnitaryLocalTs(cg2)
# ret_cg = solver_cg.run(200, disable_message=False)

number of models is :  2


In [4]:
ret_cg = solver1.run(200, disable_message=False)

dtype changes from torch.float32 to torch.float64
target loss      : 0.4143866301
initial loss     : 0.7652189180




  0%|          | 0/200 [00:00<?, ?it/s]

In [5]:
ret_cg = solver2.run(200, disable_message=False)

target loss      : 0.4143866301
initial loss     : 0.7325662710

dtype changes from torch.float64 to torch.float32
loss upper bound: 0.7465435266




  0%|          | 0/200 [00:00<?, ?it/s]

dtype changes from torch.float32 to torch.float64
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No loca

_LinAlgError: torch.linalg.eigvalsh: The algorithm failed to converge because the input matrix is ill-conditioned or has too many repeated eigenvalues (error code: 98).

In [40]:
model = copy.deepcopy(model1)
model2 = copy.deepcopy(model1)
model2.reset_params()
cg2 = RiemanUnitaryCG2([(model1, model2), (model2, model1)], [loss, loss])
cg = RiemanUnitaryCG(model, loss, lr = 0.1)
loss.target

number of models is :  2


tensor(26.6966)

In [45]:
(model2._params.grad - model1._params.grad).norm()

tensor(0., dtype=torch.float64)

In [41]:
loss_ = cg2.loss_val()
print(loss_.item()/2)
loss_.backward()
cg2.step()
print(model1._params[0])

69.04452221485724
tensor(-0.0948, dtype=torch.float64, grad_fn=<SelectBackward0>)


In [19]:
loss_ = loss(model.matrix())
print(loss_.item())
loss_.backward()
cg.step()
print(model._params[0])

5184.598771197059
tensor(-0.1200, dtype=torch.float64, grad_fn=<SelectBackward0>)


In [119]:
U = torch.kron(model1.matrix(), model3.matrix())
torch.linalg.eigvalsh(torch.abs(U.T @ loss.X @ U))

tensor([0.4727, 1.7883, 2.1820, 2.2093, 2.4626, 3.6527, 3.9013, 4.7296, 6.4704],
       dtype=torch.float64, grad_fn=<LinalgEighBackward0>)

In [124]:
cg2.loss_val([model1.matrix(), model3.matrix()])

tensor([6.4704], dtype=torch.float64, grad_fn=<AddBackward0>)

In [122]:
model1.matrix()

tensor([[-0.9294, -0.0230,  0.3683],
        [-0.3606, -0.1556, -0.9196],
        [ 0.0784, -0.9876,  0.1363]], dtype=torch.float64,
       grad_fn=<ViewBackward0>)

In [118]:
cg2.inv_list

[(tensor([[ 0.0000, -0.0925, -8.8539],
          [ 0.0925,  0.0000,  9.7535],
          [ 8.8539, -9.7535,  0.0000]], dtype=torch.float64),
  tensor([[-0.9294, -0.0230,  0.3683],
          [-0.3606, -0.1556, -0.9196],
          [ 0.0784, -0.9876,  0.1363]], dtype=torch.float64)),
 (tensor([[  0.0000,   4.0488,  -2.8550],
          [ -4.0488,   0.0000,  16.5148],
          [  2.8550, -16.5148,   0.0000]], dtype=torch.float64),
  tensor([[-0.4595,  0.6831,  0.5677],
          [-0.7013, -0.6712,  0.2401],
          [-0.5451,  0.2878, -0.7875]], dtype=torch.float64))]

In [6]:
model1.matrix()
def objective(t):
    U_list = []
    for H, U in cg2.inv_list:
        U_list.append(torch.matrix_exp(-t*H)@U)
    print(U_list)
    return cg2.loss_val(U_list)

In [7]:
objective(0)

[tensor([[-0.2220, -0.7183, -0.6588,  0.0270],
        [ 0.0659,  0.5110, -0.6044, -0.6077],
        [ 0.9316, -0.0667, -0.2299,  0.2736],
        [-0.2803,  0.4673, -0.3846,  0.7451]], dtype=torch.float64)]


tensor([10.9565], dtype=torch.float64)

In [9]:
cg2.loss_list[0]([model, model])

AttributeError: 'UnitaryRiemanGenerator' object has no attribute 'ndim'

In [19]:
for _ in range(10):
    loss(model.matrix()).backward()
    loss(model2.matrix()).backward()
    cg2.step()
    cg.step()
    cg.zero_grad()
    cg2.zero_grad()

In [20]:
model.matrix()

tensor([[ 0.6798,  0.0555,  0.4857,  0.5467],
        [ 0.1244,  0.3583,  0.5885, -0.7140],
        [-0.1170, -0.8943,  0.4119, -0.1297],
        [-0.7132,  0.2621,  0.4980,  0.4178]], dtype=torch.float64,
       grad_fn=<ViewBackward0>)

In [21]:
model2.matrix()

tensor([[ 0.6798,  0.0555,  0.4857,  0.5467],
        [ 0.1244,  0.3583,  0.5885, -0.7140],
        [-0.1170, -0.8943,  0.4119, -0.1297],
        [-0.7132,  0.2621,  0.4980,  0.4178]], dtype=torch.float64,
       grad_fn=<ViewBackward0>)

In [15]:
lr = slcg._golden(W.data, S.data)

In [16]:
lr

1.275625102976006

In [8]:
loss(model.matrix()).backward()

In [10]:
model._params.grad

tensor([  1.2769, -12.3206,   7.5504,   7.5007,  -7.3384, -47.3078, -43.3932,
         28.8005,   1.4467,  27.3550,   8.5543, -16.6534, -12.1922, -75.3265,
        -72.0942,  45.8580], dtype=torch.float64)

In [38]:
model.matrix()

tensor([[ 1.2359, -0.7446,  0.8197,  1.7623],
        [-0.4031,  0.7149,  0.1083,  0.1054],
        [-0.3327, -0.3558, -0.2180,  0.1962],
        [-1.1024, -0.0299,  0.3727, -0.9463]], dtype=torch.float64,
       grad_fn=<ViewBackward0>)

In [25]:
S

tensor([[ 1.1503e+09,  1.0250e+09],
        [-1.2910e+09, -1.1503e+09]], dtype=torch.float64)